# TFM - Parametría para la evaluación de la calidad del dato - ParametriaTFM.ipynb

### Datos del autor
Notebook elaborado por: **César Fernando Balaguer García** y **Antonio Luís Almira Martínez** en el marco de la tesis de Máster Universitario en **Análisis y Visualización de Datos Masivos / Visual Analytic y Big Data** (2025) <br>.
Asesora de tesis: **María Belén Benalcázar Tovar**. <br>
Título: **Marco Metodológico que permita medir la calidad de los datos de un conjunto de datos en formatos planos**.

### Carga las librerías necesarias para la parametría

In [1]:
import pandas as pd
import numpy as np
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import re

from IPython.display import display, clear_output
from matplotlib.backends.backend_pdf import PdfPages


### Funciones varias
En esta sección definimos funciones básicas que se usarán a lo largo de la medición de la calidad, son rutinas de apoyo que serviran entre otras cosas para imprimir los resultados finales de la medición de la calidad del conjunto de datos o la impresión de los títulos para cada sección

In [2]:
def imprimir_titulo(texto):
    print(f"\033[1m\033[4m{texto}\033[0m")  # Negrita + subrayado


In [3]:
def imprimir_parametria(configuraciones):
    imprimir_titulo("Configuraciones registradas")
    print("")
    print(f"{'Encabezado':<25} {'Dimensión':<15} {'Validador':<25} {'Umbral Min':<10} {'Umbral Acep':<10}")
    print("-" * 90)

    for config in configuraciones:
        encabezado = ", ".join(config['encabezado'])
        print(f"{encabezado:<25} {config['dimension']:<15} {config['validador']:<25} {config['umbral_minimo']:<10} {config['umbral_aceptable']:<10}")


### Configuración del validador
En esta primera parte se va a configurar el validador de la calidad del conjunto de datos, para ello se van a realizar los siguientes pasos: <br>
>- Creación de una lista con las dimensiones que se van a evaluar
>- Carga del dataset en formato CSV con la información que se va a evaluar, esto con el fin de obtener los encabezados o nombres de columnas
>- Registro de una lista de configuración, donde se tendrán los siguientes datos:
>>- Nombre de la columna a evaluar
>>- Nombre de la dimensión
>>- Validador Se registra solo para las dimensiones de Consistencia, Exactitud y Credibilidad
>>- Umbral mínimo el cual debe ser un valor numérico entre 0% y 100%
>>- Umbral máximo el cual debe ser un valor numérico entre 0% y 100%
>>
>
Finalmente la configuración almacenará el registro realizado en un archivo con extensión **CSV** para tenerlo como registro de las validaciones y umbrales realizados.

Creación de la lista de dimensiones incluyendo aquellas que se van a utilizar para la medición de la calidad y adicionalmente crea la lista de configuración, donde se registrarán los datos, dimensiones, validadores y umbrales a utilizar para medir la calidad del dato

In [4]:
dimensiones = [
    "Exactitud",
    "Completitud",
    "Consistencia",
    "Credibilidad",
    "Unicidad"
]
configuraciones = []
nombre_archivo = ""

In [5]:
nombre_archivo = input("\nIngrese el nombre del archivo de datos a evaluar (sin extensión): ")
archivo = nombre_archivo + ".csv"
df = pd.read_csv(archivo, sep=";")

encabezados = list(df.columns)
clear_output(wait=True)

imprimir_titulo("Conjunto de datos y nombre de campos a evaluar")
print("")
print(archivo)
print(encabezados)

Conjunto de datos y nombre de campos a evaluar

dataset.csv
['CONTENIDO A ANALIZAR', 'INTENSIDAD']


Solicita los datos para la configuración <br>
>- Solicita el dato <br>
>- Solicita la dimensión <br>
>- Solicita un validador <br>
>- Solicita el umbral mínimo <br>
>- Solicita el umbral aceptable.

In [6]:
imprimir_titulo("Registre los datos necesarios para realizar la validación del conjunto de datos")

while True:
    entrada = input("\nSelecciona un nombre de campo o varios (ejemplo: campo1 o [\"campo1\", \"campo2\"] a evaluar) (deja en blanco para terminar): ")
    
    if entrada.strip() == "":
        break

    try:
        encabezado = ast.literal_eval(entrada) if entrada.strip().startswith("[") else entrada.strip()
    except Exception:
        print("Formato inválido. Si deseas ingresar varios campos usa el formato: [\"campo1\", \"campo2\"]")
        continue

    if isinstance(encabezado, str):
        encabezado = [encabezado]

    if not all(campo in encabezados for campo in encabezado):
        print("Uno o más encabezados no válidos. Intenta de nuevo.")
        continue

    dimension = input("Selecciona una dimensión: ")
    if dimension not in dimensiones:
        print("Dimensión no válida. Intenta de nuevo.")
        continue

    if (dimension == "Consistencia") | (dimension == "Exactitud") | (dimension == "Credibilidad"):
        while True:
            validador = input("Ingresa el campo validador: ")
            if validador != "":
                break
            else:
                print("El campo 'validador' es obligatorio para la dimensión.")
    else:
        validador = " "

    while True:
        try:
            umbral_minimo = float(input("Ingresa el umbral mínimo (0 a 100): "))
            if 0 <= umbral_minimo <= 100:
                break
            else:
                print("Debe ser un valor entre 0 y 100.")
        except ValueError:
            print("Ingresa un número válido.")

    while True:
        try:
            umbral_aceptable = float(input("Ingresa el umbral aceptable (0 a 100): "))
            if 0 <= umbral_aceptable <= 100:
                if umbral_aceptable < umbral_minimo:
                    print("El umbral aceptable debe ser mayor o igual al umbral mínimo")
                else:
                    break
            else:
                print("Debe ser un valor entre 0 y 100.")
        except ValueError:
            print("Ingresa un número válido.")

    configuraciones.append({
        "encabezado": encabezado,
        "dimension": dimension,
        "validador": validador,
        "umbral_minimo": umbral_minimo,
        "umbral_aceptable": umbral_aceptable
    })

clear_output(wait=True)
imprimir_parametria(configuraciones)


Configuraciones registradas

Encabezado                Dimensión       Validador                 Umbral Min Umbral Acep
------------------------------------------------------------------------------------------
CONTENIDO A ANALIZAR      Completitud                               100.0      100.0     
CONTENIDO A ANALIZAR      Unicidad                                  100.0      100.0     
INTENSIDAD                Completitud                               100.0      100.0     
INTENSIDAD                Credibilidad    1, 2, 3, 4                98.0       99.0      
INTENSIDAD                Consistencia    9                         100.0      100.0     
INTENSIDAD                Exactitud       1:4                       98.0       99.0      


In [7]:
nombre_salida = nombre_archivo + ".conf"
df_config = pd.DataFrame(configuraciones)
df_config.to_csv(nombre_salida, sep=';', index=False)
clear_output(wait=True)

print(f"\nConfiguración guardada correctamente en '{nombre_salida}'")



Configuración guardada correctamente en 'dataset.conf'
